In [1]:
import gc
import pandas as pd
import numpy as np
import os

from glob import glob
from tqdm import tqdm_notebook, tqdm

pd.set_option('display.max_columns', 100)

from utils import loadpkl, to_feature, line_notify
from utils import NUM_FOLDS, FEATS_EXCLUDED, COLS_TEST1, COLS_TEST2
from utils import CustomTimeSeriesSplitter, DAYS_PRED

%matplotlib inline

In [2]:
files = sorted(glob('../feats/*.feather'))
df = pd.concat([pd.read_feather(f) for f in tqdm(files, mininterval=60)], axis=1)

  0%|          | 0/70 [00:00<?, ?it/s]/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pandas/io/feather_format.py:124: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  nthreads=int_use_threads)
/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pyarrow/pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels
100%|██████████| 70/70 [00:10<00:00,  6.74it/s]


In [6]:
df = df[df['date']>'2014-04-25']

# split train & test
train_df = df[df['date']<'2016-04-25']
test_df = df[df['date']>='2016-04-25']
del df
gc.collect()

22

In [9]:
df['day'] = df['date'].dt.day

In [6]:
CAT_COLS = ['item_id','state_id','dept_id','cat_id','wday','day','week','month',
            'quarter', 'year', 'is_weekend','snap_CA', 'snap_TX', 'snap_WI']

In [11]:
for c in CAT_COLS:
    print(c, len(df[c].value_counts()))

item_id 3049
state_id 3
dept_id 7
cat_id 3
wday 7
day 31


KeyError: 'week'

In [6]:
# set id as index
df.set_index('id', inplace=True)

# sort by date
df.sort_values('date',inplace=True)

In [10]:
df = df[df['date']>'2014-04-25']

# split train & test
#=======================================================================
# 2011-01-29 ~ 2016-04-24 : d_1    ~ d_1913
# 2016-04-25 ~ 2016-05-22 : d_1914 ~ d_1941 (public)
# 2016-05-23 ~ 2016-06-19 : d_1942 ~ d_1969 (private)
#=======================================================================

train_df = df[df['date']<'2016-04-25']
test_df = df[df['date']>='2016-04-25']

del df
gc.collect()

11

In [2]:
df